# User_antenna mapping para argentina
La idea es usar el el raw CDRs dataseel raw CDRs datase para crear la lista que cuenta antennas utilizadas por cada user diariamente y agrupar esta lista (con repeticiones) en un agregado mensual (para luego determinar home_antenna etc.)

Tambien usando info de las antenas (epidemicas o no). Podriamos ver si un user "vive" o solo "paso" por la zona epidemica.

## definiciones utilizadas

* la fecha viene dado en YYYMMDD y el TIME es HHMMSS en formato 24 horas
* *Time* columns are measured in seconds.
* Daylight corresponds to the [8 19) timeframe (using Arg's timezone)
* el analisis es solo para CDRs con usuarios de la TelCo. Los users de la TelCo van todos en la primer columna (independientemente del nombre de la columna..) entonces si la llamada es entrante quiere decir que el user de la telco esta recibiendo un llamado en esa antennaID. si la llamada es saliente es al reves. (Podria pasar que sea comunicaciones inter-Telco y ahi habria dos records uno saliente y otro entrante pero con los mismos 2 userIDs).
* La Direction, viene dada relativa al user de la primer columna (incoming ,1, si entra y viceversa si es outgoing,0).

### el dataset de simple_format vendria con este header
ORIGIN_NUMBER_ENC_B64|TARGET_NUMBER_ENC_B64|DIRECTION|CLIENT_BUSSINES_ID|OTHER_OPERATOR|FECHA|TIME|DURATION|CLIENT_CELL_ID|REDIRECT_VOICEMAIL|AIR_CHARGE|LAND_CHARGE|MONEDERO|RATE_ID_AIR|RATE_ID_LAND|DISCOUNT_AMOUNT_FF

Basicamente los primeros 2 son los hashes de los users que estuvieron en el call, con el primero el user de la telco.

Direction es quien llama a quien (outgoing es 1-->2, viceversa incoming). Fecha el dia,mes,anyo del llamado, time en formato 24hs incluyendo segundos. Duration medido en segundos, Client_cell_id el hash de la antenna y todo el resto no nos interesa.

In [38]:
%autosave 180
import pandas as pd; import numpy as np; import os;import random;
import graphlab as gl
pd.set_option('display.max_rows', 200)
#esto es para dibujar directo a la notebook
gl.canvas.set_target('ipynb')

#from matplotlib import pyplot as plt
#%matplotlib inline
import time
import os
np.random.seed(2016)

Autosaving every 180 seconds


In [2]:
#ejemplo  "/grandata/simple_format/01-02012014.txt.gz"
#seteamos el lugar de trabajo
rootdir="/grandata/ca/voice/"
os.chdir(rootdir)
year = 2015; 
months= [1,2,3];
days = range(1,32)

day = days[30]
month = months[0]
def get_input_file(month,day):
    return rootdir +"{y}/{m}/binaria_gsm_{y}{m:0=2d}{d:0=2d}.csv.gz"\
                .format(y=2012,m=month,d=day)
    
def get_output_file(month,sample=False,night_filter=False,week_end=False):
    output = "/home/juan/mobility-study/argentina-scripts/output/{0}/user_antenna_mapping".format(month)
    if week_end == True:
        output = output + "_wkend"
    if night_filter == True:
        output = output + "_ngtfilter"
    if sample == True:
        output = output + "_sample"
    return output + ".txt.gz"

In [3]:
month = 2
day = days[15]
input_file = get_input_file(month,day)

In [5]:
!zcat $input_file | wc -l
!zcat $input_file | head

60626273
ORIGIN_NUMBER_ENC_B64|TARGET_NUMBER_ENC_B64|DIRECTION|CLIENT_BUSSINES_ID|OTHER_OPERATOR|FECHA|TIME|DURATION|CLIENT_CELL_ID|REDIRECT_VOICEMAIL|AIR_CHARGE|LAND_CHARGE|MONEDERO|RATE_ID_AIR|RATE_ID_LAND|DISCOUNT_AMOUNT_FF
536C45575A4A4B38644969443735764959644E7877673D3D|675350667837583144524C6D4457592F3872337578673D3D|O|CO|C.T.I. COMPA?IA DE TELEFONOS DEL INTERIOR S.A.|20120216|103951|116|C1112A|N|0|0|GRANEL|ALOCAL|CLV06|0
41684B766F7A556176333838424B693538704A684E413D3D|6A6341334A364633755979574938714C4446493575673D3D|O|CO|TELEFONICA COMUNICACIONES PERSONALES S.A.|20120216|140811|81|SJ061A|N|.54|0|PACK|ALOCAL|TFIJO|0
69794D55646952475461714246387130466B6A3631773D3D|514D397635785854773262714F56336C4262524D2F413D3D|O|CO|CTI COMPA?IA DE TELEFONOS DEL INTERIOR S.A.|20120216|141035|363|ME002A|N|1.19|0|PACK|ALOCAL|TFIJO|0
5477557A4F795A7978717045695567497978674830513D3D|2F6A35446871587A6C397568637653785249306659513D3D|O|CO|TELEFONICA COMUNICACIONES PERSONALES S.A.|20120216|111624|24|BA

In [19]:
 pd.read_csv(input_file,sep = "|",
                   header=0,
                   engine = 'c',
                   nrows = 50,
                   error_bad_lines= False,
            warn_bad_lines = True
                  )


,ORIGIN_NUMBER_ENC_B64,TARGET_NUMBER_ENC_B64,DIRECTION,CLIENT_BUSSINES_ID,OTHER_OPERATOR,FECHA,TIME,DURATION,CLIENT_CELL_ID,REDIRECT_VOICEMAIL,AIR_CHARGE,LAND_CHARGE,MONEDERO,RATE_ID_AIR,RATE_ID_LAND,DISCOUNT_AMOUNT_FF
0,536C45575A4A4B38644969443735764959644E7877673D3D,675350667837583144524C6D4457592F3872337578673D3D,O,CO,C.T.I. COMPA?IA DE TELEFONOS DEL INTERIOR S.A.,20120216,103951,116,C1112A,N,0.0000,0,GRANEL,ALOCAL,CLV06,0
1,41684B766F7A556176333838424B693538704A684E413D3D,6A6341334A364633755979574938714C4446493575673D3D,O,CO,TELEFONICA COMUNICACIONES PERSONALES S.A.,20120216,140811,81,SJ061A,N,0.5400,0,PACK,ALOCAL,TFIJO,0
2,69794D55646952475461714246387130466B6A3631773D3D,514D397635785854773262714F56336C4262524D2F413D3D,O,CO,CTI COMPA?IA DE TELEFONOS DEL INTERIOR S.A.,20120216,141035,363,ME002A,N,1.1900,0,PACK,ALOCAL,TFIJO,0
3,5477557A4F795A7978717045695567497978674830513D3D,2F6A35446871587A6C397568637653785249306659513D3D,O,CO,TELEFONICA COMUNICACIONES PERSONALES S.A.,20120216,111624,24,BA740D,N,0.1600,0,PACK,ALOCAL,TFIJO,0
4,6936563854705241336773394B34493142496F5671513D3D,326C52694447393463755A704654354362506E6279513D3D,O,CO,TELEFONICA COMUNICACIONES PERSONALES S.A.,20120216,122953,21,CB205I,N,0.2009,0,PACK,ALOCAL,TFIJO,0
5,5769454D4D4D5738706A326D4E59786441387A6131673D3D,5769454D4D4D5738706A304E71594E4A7333545145413D3D,O,CO,CTI COMPA?IA DE TELEFONOS DEL INTERIOR S.A.,20120216,122434,10,SF023C,N,0.0000,0,GRANEL,ALOCAL,TFIJO,0
6,6238566132392F65756B4266594D6876326439666A413D3D,7674386B594E54636D6B506373374F70374C416649673D3D,O,CO,TELEFONICA COMUNICACIONES PERSONALES S.A.,20120216,121553,13,CF117Z,N,0.1693,0,PACK,ALOCAL,TFIJO,0
7,666577306654646A6E62323534536B723738633547413D3D,754D35387053454833624D5977654D736747594833773D3D,O,CO,TELEFONICA COMUNICACIONES PERSONALES S.A.,20120216,121812,11,BA602A,N,0.2200,0,PACK,ALOCAL,TFIJO,0
8,45712F593645674737434268446D76637032646E4B513D3D,7654544B586E6C515550585A4A306D69376A616C6D773D3D,O,CO,COMPA?IA DE RADIOCOMUNICACIONES MOVILES S.A.,20120216,122723,9,BA055E,N,0.2200,0,PACK,ALOCAL,CLV09,0
9,5A55495A69696C645939396F677035316D46474C7A513D3D,5952666578564E374673756D447873787278717A67413D3D,O,CO,TELECOM PERSONAL S.A.,20120216,123040,2,CF061B,N,0.3754,0,PACK,ALOCAL,TFIJO,0


## Analytics on files
Hay masomenos 60m de lineas x dia, i.e. 60m de llamados

## Read Raw files

In [ ]:
%%time
test = pd.read_csv(input_file,sep = "|",
                   header=0,
                   engine = 'c',
                   #nrows = 5*10**6,
                   usecols = ['ORIGIN_NUMBER_ENC_B64','TARGET_NUMBER_ENC_B64','DIRECTION',
                              'FECHA','TIME','DURATION','CLIENT_CELL_ID'],
                dtype = {'TIME':np.uint32,'FECHA':np.uint32,  'CLIENT_CELL_ID':np.object_, 
                         'DURATION':np.uint16,'ORIGIN_NUMBER_ENC_B64':np.object_,'TARGET_NUMBER_ENC_B64':np.object_},
                   error_bad_lines= False,
            warn_bad_lines = True
                  )

test.rename(columns ={'FECHA':'DATE','ORIGIN_NUMBER_ENC_B64':'USER',
                      'TARGET_NUMBER_ENC_B64':'OTHER_USER'},inplace=True)


In [8]:
%%time
test['DIRECTION'] = test['DIRECTION'].str.replace('O','0').replace('I','1').astype(np.uint8)

In [16]:
test.sample(10)

,USER,OTHER_USER,DIRECTION,DATE,TIME,DURATION,CLIENT_CELL_ID
18985524,736D723753625056597257617736532B4A72583746513D3D,736D72375362505659725637696A614250564B796A673D3D,0,20120216,175831,20,CTI 17
13442718,756B3570786C584D4C4B4979436853334F384A4F56413D3D,66613547702F4F4A74317138516B426868644D6E68773D3D,0,20120216,194248,47,CO828C
52306801,364B577271545447764F3474494E574A6762307751673D3D,343874495772357263505A65676C663258354F4472773D3D,1,20120216,3815,1926,CTI 40
16751144,4D664D486E4C4E644E62784B7048715A576C354B72673D3D,6466682B6E786B58324A4978504F41762B624D4850773D3D,0,20120216,192018,8,CF628C
56824898,352F616553313962516C4C71396F46386B666B7556673D3D,7934662B6430464C5A774842376C6B7A643449655A513D3D,1,20120216,134334,2,CTI 41
15688710,6F2B335033655637684465335A7855426C53466462673D3D,4C32796776446556334F5A31793064466C51653247773D3D,0,20120216,2546,329,C1105C
6382002,4C78363839425146735964455368516E456A32482F773D3D,4B363151414A4279694C754B31474C7148536F7A69513D3D,0,20120216,160839,101,CTI 8
39859642,6A666A683079724D794D3162594D667A4D2B2B6F6D773D3D,5A4E486A634A535554716E696D536475346C4C6A6C673D3D,1,20120216,200809,3,CTI 13
58901785,364750756E32496764795163684F45304755704963673D3D,697650435668766B64334270734F54636F6A327674513D3D,1,20120216,113831,29,CTI 24
23288563,4A4B63587A596771643866564656624930317A7841773D3D,696671677271464F31314D61563332516B30744376513D3D,1,20120216,162631,61,CTI 16


## Probamos con GraphLab gl
tiene buenas conexiones con Spark, es mas rapido para los applys (ya probado) y parece ser mejor en terminos de que puede levantar files mas grandes que el tamanyo que tienen en la memoria. 

Ademas nativamente paraleliza todo con cython. Mirar htop como impacta cuando usamos 'applys' sobre el dataset.

En gl pesa 3GB lo que con pd pesaba 10GB!!

In [21]:
import datetime


'2012-02-16'

In [4]:
%%time
#columnt_type_hints=[str, str, str, str, str,int ,int ,int ,
#                                      str, str, float,float,str,str,str,int ] 

#obs. NO levanto la columna Fecha pues esta info la puedo sacar del filename
current_date = datetime.datetime(2012,month,day)

test = gl.SFrame.read_csv( input_file, delimiter='|', 
        header=True, skip_initial_space=True, 
        column_type_hints = [str, str, str  , int, str], 
        na_values=['NaN'], 
        usecols = ['ORIGIN_NUMBER_ENC_B64','DIRECTION',
                            'TIME','DURATION','CLIENT_CELL_ID'], 
        nrows=None, 
        error_bad_lines=False
                  )

test['DIRECTION'] = test['DIRECTION'].apply(lambda x: 0 if x=='O' else 1)

test.rename({'ORIGIN_NUMBER_ENC_B64':'USER',
                      'TARGET_NUMBER_ENC_B64':'OTHER_USER'})

/home/juan/mfixman/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:315: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/home/juan/mfixman/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:120: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
2016-05-25 19:26:02,969 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_14

This non-commercial license of GraphLab Create is assigned to svega@dm.uba.ar and will expire on April 16, 2017. For commercial licensing options, visit https://dato.com/buy/.


Read 278954 lines. Lines per second: 99946.8

Read 1670061 lines. Lines per second: 211109

Read 3063846 lines. Lines per second: 232266

...

Read 58343191 lines. Lines per second: 271639

Read 59793801 lines. Lines per second: 271825

Finished parsing file /grandata/ca/voice/2012/2/binaria_gsm_20120216.csv.gz

Parsing completed. Parsed 60626272 lines in 222.521 secs.

CPU times: user 15min 22s, sys: 48 s, total: 16min 10s
Wall time: 3min 48s


In [16]:
test.head()

USER,OTHER_USER,DIRECTION,DATE,TIME,DURATION,CLIENT_CELL_ID
536C45575A4A4B38644969443735764959644E7877673D3D ...,675350667837583144524C6D4457592F3872337578673D3D ...,0,20120216,103951,116,C1112A
41684B766F7A556176333838424B693538704A684E413D3D ...,6A6341334A364633755979574938714C4446493575673D3D ...,0,20120216,140811,81,SJ061A
69794D55646952475461714246387130466B6A3631773D3D ...,514D397635785854773262714F56336C4262524D2F413D3D ...,0,20120216,141035,363,ME002A
5477557A4F795A7978717045695567497978674830513D3D ...,2F6A35446871587A6C397568637653785249306659513D3D ...,0,20120216,111624,24,BA740D
6936563854705241336773394B34493142496F5671513D3D ...,326C52694447393463755A704654354362506E6279513D3D ...,0,20120216,122953,21,CB205I
5769454D4D4D5738706A326D4E59786441387A6131673D3D ...,5769454D4D4D5738706A304E71594E4A7333545145413D3D ...,0,20120216,122434,10,SF023C
6238566132392F65756B4266594D6876326439666A413D3D ...,7674386B594E54636D6B506373374F70374C416649673D3D ...,0,20120216,121553,13,CF117Z
666577306654646A6E62323534536B723738633547413D3D ...,754D35387053454833624D5977654D736747594833773D3D ...,0,20120216,121812,11,BA602A
45712F593645674737434268446D76637032646E4B513D3D ...,7654544B586E6C515550585A4A306D69376A616C6D773D3D ...,0,20120216,122723,9,BA055E
5A55495A69696C645939396F677035316D46474C7A513D3D ...,5952666578564E374673756D447873787278717A67413D3D ...,0,20120216,123040,2,CF061B


In [10]:
%%time
#paso las fechas a un timestamp
test['TIMESTAMP'] = test['DATE','TIME'].apply(str(current_date).split(" ")[0]
                                 +'-'+x['TIME'][0:2]+'-'+x['TIME'][2:4]+'-'+x['TIME'][4:6])

CPU times: user 24 ms, sys: 28 ms, total: 52 ms
Wall time: 47.6 ms


In [11]:
%%time
test['TIMESTAMP'] = test['TIMESTAMP'].str_to_datetime(str_format='%Y-%m-%d-%H:%M:%S')

CPU times: user 7min 46s, sys: 18.4 s, total: 8min 4s
Wall time: 1min 15s


In [26]:
%%time
mini_sample = test.sample(1e-6).copy()

CPU times: user 18.2 s, sys: 3.57 s, total: 21.7 s
Wall time: 8.35 s


In [32]:
int(current_date.weekday()==5 or current_date.weekday()==6)

0

In [33]:
## testeo si las llamadas fueron hechas durante el findesemana
test['DURING_WEEKEND'] = int(current_date.weekday()==5 or current_date.weekday()==6)

In [51]:
%%time
# asigno el bool si la llamada fue en horario 'laboral'
#notar que la hora < 19 significa que 18hs y 59 min da True
test['DURING_DAYLIGHT']= test['TIMESTAMP'].apply(lambda x: x.hour>=8 and x.hour<19 )

CPU times: user 4 ms, sys: 8 ms, total: 12 ms
Wall time: 9.22 ms


In [52]:
%%time
mini_sample = test.sample(1e-4).copy()

CPU times: user 2.04 s, sys: 12 ms, total: 2.05 s
Wall time: 2.04 s


In [55]:
mini_sample.column_names()

['USER',
 'OTHER_USER',
 'DIRECTION',
 'DATE',
 'TIME',
 'DURATION',
 'CLIENT_CELL_ID',
 'TIMESTAMP',
 'DURING_WEEKEND',
 'DURING_DAYLIGHT']

In [61]:
mini_sample.shape

((6030, 2), (6055, 10))

In [69]:
%%time
a = test.groupby(['USER','CLIENT_CELL_ID'],
                 {'ANTENNA_COUNT':gl.aggregate.COUNT()})

CPU times: user 20min 23s, sys: 26.6 s, total: 20min 49s
Wall time: 5min 22s


In [70]:
a.head()

CLIENT_CELL_ID,USER,ANTENNA_COUNT
CTI 13,4A4F527557786A55676F4232614549395136576463413D3D ...,2
CO271K,547544415041724F4F545A4A656D41773865676B54773D3D ...,1
CTI 1,694343555935704D326C3038745248614E70424653413D3D ...,4
NQ213F,6B646A49327A5177714F526463696736566C4C4A64413D3D ...,2
CO720A,57486F4C633635447152767A3933595468544F2F36673D3D ...,1
SF893A,763832655244745265534246784B3469576765496B413D3D ...,1
071,636D592F5631474546686F6A4C655A315A4D395245413D3D ...,1
SF136B,4752342F614E7A5A54703044336F6D474770694851513D3D ...,1
CF001X,474336483069486A45343134616158534574472F45413D3D ...,1
SF231A,796A4850656475654441686F464557355837507548513D3D ...,1


In [71]:
a.shape

(22669212, 3)

In [72]:
%%time
a = a.groupby(['USER'],
                 {'ANTENNA_COUNT_DICT':gl.aggregate.CONCAT("CLIENT_CELL_ID","ANTENNA_COUNT")})

CPU times: user 12min 33s, sys: 20.4 s, total: 12min 53s
Wall time: 3min 4s


In [73]:
a.shape

(8501920, 2)

In [74]:
%%time
#chequeamos que no estemos perdiendo users.
len(test['USER'].unique())

CPU times: user 9min 16s, sys: 8.6 s, total: 9min 25s
Wall time: 1min 19s


8501920

In [246]:
mini= a.sample(1e-5)

In [248]:
def get_columns_from_dict(user_dict):

    antenna_count_list = [-1]*10
    #por un problema de type consistency dentro de la listas que le meto al SFrame, aca los -1 tienen que ser strings
    antenna_list = ["-1"]*10
    sorted_user_dict = sorted(user_dict, key=b.get, reverse=True)
    for i,ant in enumerate(sorted_user_dict):
        if(i>9):
            break
        antenna_count_list[i] = user_dict[ant]
        antenna_list[i] =  ant
            #print(user_dict)
        #for other than the first case
        if i==0:
            continue
        #random permutation of current and previous antenna    
        if user_dict[sorted_user_dict[i-1]]==user_dict[ant]  and np.random.random() <= 0.5:
            #add past antennas data to current
            antenna_count_list[i] = antenna_count_list[i-1]
            antenna_list[i] =  antenna_list[i-1]
            ## add current data to past
            antenna_count_list[i-1] = user_dict[ant]
            antenna_list[i-1] =  ant
    
    rv = (antenna_list, antenna_count_list)
    return rv
            

In [260]:
%%time
a['ANTENNA_ID'] = a['ANTENNA_COUNT_DICT'].apply(lambda user: get_columns_from_dict(user))
a['COUNT'] = a['ANTENNA_ID'].apply(lambda row: row[1])
a['ANTENNA_ID'] = a['ANTENNA_ID'].apply(lambda row: row[0])
a = a.unpack('ANTENNA_ID') 
a = a.unpack('COUNT') 
a = a.rename(dict([(col,col.replace(".","_")) for col in a.column_names() if ("." in col)]))
finished_counting = True

CPU times: user 108 ms, sys: 68 ms, total: 176 ms
Wall time: 181 ms


In [266]:
## falta dropear el column dict nomas

In [37]:
test['DURATION'].show(view='Numeric')

In [31]:
mini_sample['DATE','TIME']

DATE,TIME
20120216,103951
20120216,140811
20120216,141035
20120216,111624
20120216,122953
20120216,122434
20120216,121553
20120216,121812
20120216,122723
20120216,123040


In [33]:
mini_sample['DATE','TIME']

dtype: str
Rows: 15
['2012-02-16-10-39-51', '2012-02-16-14-08-11', '2012-02-16-14-10-35', '2012-02-16-11-16-24', '2012-02-16-12-29-53', '2012-02-16-12-24-34', '2012-02-16-12-15-53', '2012-02-16-12-18-12', '2012-02-16-12-27-23', '2012-02-16-12-30-40', '2012-02-16-12-22-26', '2012-02-16-12-31-47', '2012-02-16-90-72-0', '2012-02-16-13-24-02', '2012-02-16-13-16-27']

In [4]:
#para enriquecer el dataset de CDRs con info acerca de la zona epidemica etc.
antennas= pd.read_csv('/home/juan/mobility-study/antennas_mexico.csv',sep = "|",header=0,index_col=0)
antennas.index.rename("ANTENNA_ID",inplace=True)

In [6]:
sample_file = input_file.replace(".txt.gz","_sample.txt.gz")
#para el manejo de las distintas salidas del programa


week_end = False
night_filter = False
hash_map = False
output_file = get_output_file(hash_map,night_filter,week_end)

In [ ]:
sample.to_csv(sample_file,
              sep=" ",
              header=False,
              index=False,
             )

## testeo el algo en un file pequenyo

In [8]:
#leo las primeras 10M lineas



sample = pd.read_csv(
                input_file,
                engine = 'c',
#                chunksize = 10**7,
    #            iterator =True,
                nrows = 10**7,
                sep = ' ',
                header = 0,
                index_col=None,
                usecols = ['USER','OTHER_USER','ANTENNA_ID','TIMESTAMP'],
                dtype = {'TIMESTAMP':np.uint32,'ANTENNA_ID':np.uint16,'USER':np.object_,'OTHER_USER':np.object_}
                )


In [28]:
## script para extraer todos los atributos del simple_format

#ver el tiempo que tarda
start_time = time.time()

#el chunk basicamente va leyendo el file de a 'chunksize' cantidad de filas
#subgroup = pd.DataFrame()

#hay cinco meses del cual extraer atributos:
months= ["0"+str(month) for month in range(5,10)]

year = "2015"

#aca seteamos como vamos a partir la tabla segun el nro correspondiente a c/hash y tomando modulo
passes = 45
#creo los grupos que despues van a filtrar c/chunk de la tabla para hacer varias pasadas
for group in range(passes):
    print("working group number {it} of {pas}, time elapsed is {t} \n".format(it=group,pas=passes, t=(time.time()-start_time)))

    #itero sobre los meses
    subgroup = pd.DataFrame()
    for month in months:
        print("Working on month {ms}-{y}, time elapsed is {t} ".format(ms=month,y=year,t=(time.time()-start_time)))
        input_file= rootdir +"simple_format_{y}{ms}.txt.gz"\
            .format(y=year,ms=month)

        table = pd.read_csv(
                input_file,
                engine = 'c',
                chunksize = 6*10**7,
    #            iterator =True,
                sep = ' ',
                header = 0,
                index_col=None,
                usecols = ['USER','OTHER_USER','ANTENNA_ID','TIMESTAMP'],
                dtype = {'TIMESTAMP':np.uint32,'ANTENNA_ID':np.uint16,'USER':np.object_,'OTHER_USER':np.object_}
                )

        #cuando entramos a este loop, table tiene tantos 'chunks' como el valor entero de la cantidad de lineas en el file
        #dividido el tamanyo del chunksize

        numb=0;
        for chunk in table:
                numb+=1
                #a cada chunk filtro por todos los USERs por el hash del string (el hash es un int) y despue filtro modulo passes
                #y trabajo sobre la tabla subgroup nada mas que ahora tiene menos usuarios
                subgroup = subgroup.append(chunk[chunk['USER'].apply(lambda x: hash(x) % passes == group )])

    #entonces la idea es que yo ahora solo voy a trabajar, dentro de esta tabla filtrada y para todos los meses juntos
    
    #paso de segundos a horas
    #notar que TIMESTAMP arranca en 0 segundos para domingo 01/01/2012 00:00am 
    #con lo cual domingo es el dia 0, lunes el 1, asi..
    print('finished month reading')
    subgroup['Hour'] =  (subgroup['TIMESTAMP'].values*1.0/3600)%24
    
    subgroup['Day']  =  (subgroup['TIMESTAMP'].values*1.0/(3600*24))%7

    #filtro usuarios con pocos o demasiados llamados en general menos de 25 y mas de 2000 en 5 meses  
    insignificant_users = subgroup['USER'].value_counts()[(subgroup['USER'].value_counts() < 25) \
                        | (subgroup['USER'].value_counts() > 2000)].index.values.tolist()
    subgroup= subgroup.loc[~subgroup['USER'].isin(insignificant_users)]


    #filtro segun nightfilter y week_end. el day light seria de [7,19) segun la convencion de mexico/GranData
    if night_filter == True:
        subgroup = subgroup.loc[(subroup['Hour']<7) | (subgroup['Hour']>19)]
    if week_end == True:
        subgroup = subgroup.loc[(subgroup['Day']==0) | (subgroup['Day']==6)]


    #   

    grouped = subgroup.groupby(['USER', 'ANTENNA_ID'])['ANTENNA_ID'].agg({'count': np.size})
    grouped.reset_index(inplace=True,drop=False)

    del subgroup
    #reordeno dentro de c/ USER por el count del antenna, esto me sirve para despues ordenar las antenas por uso
    grouped.sort_values(by=['USER','count'],ascending=False,inplace=True)

    ##enriquezco la muestra con datos epidemicos
    #primero agrego a cada antenna del df el dato de si es epidemica
    #despues agrupo por el USER y me fijo solo la columna epidemica en c/grupo
    #finalmente sumo en c/ grupo y tomo la parte superior 
    #entera de esa division con el largo del grupo. Si uso al menos una antena epidemica entonces esta expuesto(==1) Si no,
    # da 0 pues no estuvo expuesto.        
    ##enriquezco la muestra con datos epidemicos

    exposed_info =grouped.join(antennas['EPIDEMIC'], on='ANTENNA_ID').\
    groupby('USER')['EPIDEMIC'].\
        agg({'EXPOSED' : lambda x: int( np.ceil(np.sum(x)*1.0/np.size(x)) )})

    #actualizo la tabla
    grouped = grouped.join(exposed_info['EXPOSED'],on="USER")
    del exposed_info


    #creo la tabla filtrada solo por users, que es la que voy a terminar guardando (hay tantos rows como users)
    output_table = grouped.drop_duplicates(subset = 'USER', keep='first')
    #re indexo
    output_table.index = output_table['USER'].values

    #agrupo ahora la tabla por USER para hacer todos los calculos en los grupos

    grouped = grouped.groupby('USER')

    #aca voy a ir agregando las top 10 antennas utilizadas por el user, Si no llego a 10 antennas, relleno con NaNs
    for i in range(10):
        #me quedo con la i-esima fila de c/grupo (si no hay fila, no toma en cuenta ese USER)

        buffer_table = grouped.nth(i)[['ANTENNA_ID','count']]
        #renombre a iesima ANTENNA_ID e iesimo count
        buffer_table.columns=['ANTENNA_ID_%i'%i,'count_%i'%i]
        #agrego esta info como nuevas columnas, dejando lo demas como NaNs
        output_table = pd.concat([output_table, buffer_table], axis=1, join_axes=[output_table.index])

    del grouped
    #dropeo la columna USER que ya es redundante por el indice
    output_table.index.name = "USER"
    output_table.drop('USER', axis=1, inplace=True)
    
    #los primeros ANTENNA_IDs ya no me sirven, idem con el primer
    output_table.drop('ANTENNA_ID', axis=1, inplace=True)
    output_table.drop('count', axis=1, inplace=True)
    
    

    #para los datos faltantes dentro del Top10, relleno con -1s, que vendrian a ser los NaNs
    output_table.fillna(-1,inplace=True)

    #como elimine la columan de hashes puedo convertir todo a int
    output_table = output_table.astype(int,copy=False)

    #agrego info de EPIDEMIC, asumiendo a alguien como epidemico si al antenna donde vive (la ANTENNA_ID_0) 
    #esta catalogada como EPIDEMIC

    output_table = output_table.join(antennas['EPIDEMIC'], on='ANTENNA_ID_0')

    #aca termino guardando (en forma de append) el output final pero solo para esos usuarios % pass ==group

    #el primer write (first group==0) va con header
    if group ==0 :
        output_table.to_csv(output_file, index = True,
                   header = True,
                    mode='a',
                compression='gzip')
    else: 
        output_table.to_csv(output_file, index = True,
                   header = False,
                    mode='a',
                    compression='gzip')
        
    print("Finished group {gr} of {ps}, time elapsed is {t}\n ".format(gr=group,ps=passes,t=(time.time()-start_time)))
then = start_time
seconds = time.time() - then
print("total running time of script is %d " % seconds)

working group number 0 of 45, time elapsed is 0.00221490859985 

Working on month 05-2015, time elapsed is 0.0901749134064 


CParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

## cambiamos el working dir y el input_file (output.txt.gz --> output1.txt.gz)
Esto pues vamos salvando los resultados del procesamiento de datos en files sucesivos para no tener que repetir el algoritmo ni para arriesgar cambiar una version (en terminos de datos) y no tener checkpoints intermedios

In [10]:
#ejemplo  "/grandata/simple_format/01-02012014.txt.gz"
#seteamos el lugar de trabajo
rootdir="/home/juan/mobility-study/"
os.chdir(rootdir)
year = "2015"; 
month_start= "01";
#todos estos sum_links terminan 2 meses mas tardes
month_end= str(int(month_start)+2)
if int(month_end )<10:
    month_end="0"+month_end
    
day_start = "01"
day_end = str(int(day_start)+1)
if int(day_end )<10:
    day_end="0"+ day_end

input_file= rootdir +"output1.txt.gz"\
                .format(y=year,ms=month_start,me=month_end,
                       ds=day_start,de=day_end)
output_file = "/home/juan/mobility-study/output2"
if week_end == True:
    output_file = output_file + "_wkend"
if night_filter == True:
    output_file += "_ngtfilter"
if hash_map == True:
    output_file += "_user_hash_map"
output_file += ".txt.gz"

In [11]:
!echo $input_file

/home/juan/mobility-study/output1.txt.gz


In [12]:
!du -ha $input_file

165M	/home/juan/mobility-study/output1.txt.gz


In [13]:
!zcat $input_file | head -n20

USER,EXPOSED,ANTENNA_ID_0,count_0,ANTENNA_ID_1,count_1,ANTENNA_ID_2,count_2,ANTENNA_ID_3,count_3,ANTENNA_ID_4,count_4,ANTENNA_ID_5,count_5,ANTENNA_ID_6,count_6,ANTENNA_ID_7,count_7,ANTENNA_ID_8,count_8,ANTENNA_ID_9,count_9,EPIDEMIC
FFFF78CB080636632B2D1DE7A7BFAB03,0,3415,21,2969,5,1320,3,2471,3,1115,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
FFFF51FDDD55A2E24D074DE30C4798E9,1,1207,15,1171,14,1820,9,583,8,2026,5,1066,4,1831,4,1625,2,401,1,825,1,0
FFFE1EDDA25369CBC467536FA6A787FD,1,1847,22,1753,14,1422,11,1028,10,765,9,3072,9,862,8,2463,7,1416,6,871,5,1
FFFD0A912A959CDFCFFAB2F93BFBA435,1,1407,18,1702,8,1613,6,1704,5,1394,3,1398,3,818,2,958,2,726,1,848,1,0
FFFCB235D6D917DDF350F5B11CCB0EE8,1,17,74,174,33,1123,4,1178,4,173,3,890,2,503,1,1039,1,1228,1,1327,1,1
FFFC9EC4F662C44D3006BA69C9F0DE85,1,876,19,1737,6,1744,5,326,1,378,1,503,1,1118,1,2216,1,2477,1,2962,1,1
FFFB2F18F4CCF253C06B581EA518A7A5,1,2543,13,704,7,2540,7,1453,6,1642,5,45,4,2393,4,1841,3,3031,3,1009,2,0
FFFA147CB2B23AD12F207F0EFBEB14D0,1,

In [14]:
table = pd.read_csv(
                input_file,
                engine = 'c',
                sep = ',',
                header = 0,
                index_col=0,
                dtype = {'EXPOSED':np.bool,'EPIDEMIC':np.bool,
                         'ANTENNA_ID_0':np.int16,'count_0':np.uint16,'ANTENNA_ID_1':np.int16,'count_1':np.uint16,
                         'ANTENNA_ID_2':np.int16,'count_2':np.uint16,'ANTENNA_ID_3':np.int16,'count_3':np.uint16,
                        'ANTENNA_ID_4':np.int16,'count_4':np.uint16,'ANTENNA_ID_5':np.int16,'count_5':np.uint16,
                        'ANTENNA_ID_6':np.int16,'count_6':np.uint16,'ANTENNA_ID_7':np.int16,'count_7':np.uint16,
                        'ANTENNA_ID_8':np.int16,'count_8':np.uint16,'ANTENNA_ID_9':np.int16,'count_9':np.uint16}
                )
## ojo con setear los dtypes para las antennas en np.usigned int pues ahi me transforma todos los -1s en el max del dtype

In [15]:
col_labels = [ col.replace("count","COUNT") if ("count" in col) else col for col in table.columns]
table.columns = col_labels

In [16]:
#df['new_col'] = zip(df.lat, df.long)
antenna_lat_long_dict = dict(zip(antennas.index.values, zip(antennas.LATITUDE,antennas.LONGITUDE)))

In [17]:
import math
#earht_radi in kms
earth_radi = 6371

def distance_on_unit_sphere(lat1, long1, lat2, long2, earth_radi=6371):
 
    # Convert latitude and longitude to 
    # spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0
         
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
         
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
         
    # Compute spherical distance from spherical coordinates.
         
    # For two locations in spherical coordinates 
    # (1, theta, phi) and (1, theta', phi')
    # cosine( arc length ) = 
    #    sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length
     
    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) + 
           math.cos(phi1)*math.cos(phi2))
    arc = math.acos( cos )
 
    # Remember to multiply arc by the radius of the earth 
    # in your favorite set of units to get length.
    return arc*earth_radi

In [18]:
from scipy import spatial

### checking antenna_id consistency
para ver si todos los antenna_ids de nuestra tabla estan en la base de datos de las antennas.

In [17]:
def test_antennaid_consistency(row,antenna_id_list):
    res = False
    col_labels = table.columns
    #miramos solo las columnas de antenna_ids 
    row = row[[col for col in table.columns if ("ANTENNA" in col)]]
    #filtramos aquellas que tienen ant_id no nulo (==-1)
    row = row[row >-1]
    #print(row)
    #le pregunto a la serie si c/valor esta en el id_list, paso a un ndarray y consulto si todos estan
    res = row.isin(antenna_id_list).values.all()
    
    return res

In [20]:
antenna_id_list = list(antennas.index.values)

In [21]:
%%time
consistency = table.sample(10**5).apply(func = (lambda row: test_antennaid_consistency(row,antenna_id_list)),axis='columns')
#testeo que todos los resultados sean True
print(consistency.values.all())
del consistency


False
CPU times: user 2min 18s, sys: 76 ms, total: 2min 18s
Wall time: 2min 18s


In [141]:
user = 'FFFA147CB2B23AD12F207F0EFBEB14D0'
row = table.loc[user]
row = row[row>-1]
row = row[[ind for ind in row.index if "ANTENNA" in ind]]

### END antenna consistency test

In [22]:
##algoritmo sin convex hull

filter_col = [col for col in list(table) if col.startswith('ANTENNA')]

def convex_hull_radi(row,antenna_lat_long_dict,filter_col):
    #rows are pandas series
    #print(row.name)
    row =  row[filter_col]
    latitudes=[]
    longitudes =[]
    
    #cuidado con los NaNs resultantes del map
    for ant_id in (row.map(antenna_lat_long_dict).dropna()):
        latitudes += [ant_id[0]] 
        longitudes += [ant_id[1]]
    
    
    #try:
    radii=0
    for i in range(len(latitudes)):
        for j in range(i):
            new_dist= distance_on_unit_sphere(latitudes[i], longitudes[i], latitudes[j], longitudes[j])
            if radii< new_dist:
                radii = new_dist 

    return radii
    #except:
    #    print(row.name)
    #    return 0
        
       
#row =test_table.loc[2][filter_col]
#latitudes=[]
#longitudes =[]
#for ant_id in row.map(antenna_lat_long_dict).dropna():
#    latitudes += [ant_id[0]] 
#    longitudes += [ant_id[1]]
#latitudes

#convex_hull_radi(test_table.loc[12],antenna_lat_long_dict,filter_col)

In [23]:
## algoritmo con convexhulll

filter_col = [col for col in list(table) if col.startswith('ANTENNA')]

def convex_hull_radi(row,antenna_lat_long_dict,filter_col):
    #rows are pandas series
    #print(row.name)
    row =  row[filter_col]
    latitudes=[]
    longitudes =[]
    
    #cuidado con los NaNs resultantes del map
    for ant_id in (row.map(antenna_lat_long_dict).dropna()):
        latitudes += [ant_id[0]] 
        longitudes += [ant_id[1]]
    #case: only 1 antennas
    if len(latitudes)<=1:
        return 0
    
  
    
    #case only 2 antennas, arc distance
    elif len(latitudes)==2:
        return distance_on_unit_sphere(latitudes[0],longitudes[0],latitudes[1],longitudes[1])
    
    
    elif  len(latitudes)>2:
        points = np.array([np.asarray(latitudes).transpose() ,np.asarray(longitudes).transpose()])
        points = points.transpose()
        
        #chequeamos los casos super patologicos que todas las antenas esten sobre la misma linea
        if latitudes.count(latitudes[0]) == len(latitudes):
            radii=0

            for i in range(len(longitudes)-1):
                new_dist= distance_on_unit_sphere(latitudes[0], longitudes[i], latitudes[0], longitudes[i+1])
                if radii< new_dist:
                    radii = new_dist 
        
        #chequear sobre la otra linea
        elif longitudes.count(longitudes[0]) == len(longitudes):    
            radii=0
            for i in range(len(latitudes)-1):
                new_dist= distance_on_unit_sphere(latitudes[i], longitudes[0], latitudes[i+1], longitudes[0])
                if radii< new_dist:
                    radii = new_dist 
        else:
            try:
                chull = spatial.ConvexHull(points)
                #points = spatial.ConvexHull(points).points
                points = chull.points
            except:
                print(row.name)
            vert = [list(point) for point in chull.points] #chull vertices

            radii=0
            for i in range(len(vert)):
                for j in range(i):
                    new_dist= distance_on_unit_sphere(vert[i][0], vert[i][1], vert[j][0], vert[j][1])
                    if radii< new_dist:
                        radii = new_dist 

        return radii
        #return ("%s,len(lats) is %d" %(radii,len(latitudes)))

#convex_hull_radi(test_table.loc[12],antenna_lat_long_dict,filter_col)

In [ ]:
start_time = time.time()
#mini_sample =  table.sample(10**3)
table['MOBILITY_DIAMETER']= table.apply(lambda row: convex_hull_radi(row,antenna_lat_long_dict,filter_col),axis=1)

print("elapsed time is %s seconds for %d amount of rows" % (time.time()-start_time,len(mini_sample)))

In [309]:
antennas.loc[1695].LONGITUDE, antennas.loc[1311].LONGITUDE,antennas.loc[1179].LONGITUDE

(-99.191000000000003, -99.191000000000003, -99.191000000000003)

In [317]:
table.columns

Index([u'EXPOSED', u'ANTENNA_ID_0', u'COUNT_0', u'ANTENNA_ID_1', u'COUNT_1',
       u'ANTENNA_ID_2', u'COUNT_2', u'ANTENNA_ID_3', u'COUNT_3',
       u'ANTENNA_ID_4', u'COUNT_4', u'ANTENNA_ID_5', u'COUNT_5',
       u'ANTENNA_ID_6', u'COUNT_6', u'ANTENNA_ID_7', u'COUNT_7',
       u'ANTENNA_ID_8', u'COUNT_8', u'ANTENNA_ID_9', u'COUNT_9', u'EPIDEMIC',
       u'MOBILITY_DIAMETER'],
      dtype='object')

## un par de corridas usando el diametro del convex/hull dan 
elapsed time is 253.765459061 for 40000 amount of rows

elapsed time is 249.712596893 seconds for 40000 amount of rows

elapsed time is 227.963900089 seconds for 40000 amount of rows

elapsed time is 57.2980730534 seconds for 10000 amount of rows

elapsed time is 0.896952867508 seconds for 10 amount of rows



### Testeamos la validez de resultados
Nos fijamos en todos los users que tengan al menos dos antenas, en los users que solo tienen 1 antenna, etc.

In [249]:
mini_sample[(mini_sample['MOBILITY_DIAMETER']<10**3) & (mini_sample['ANTENNA_ID_0']>-1)].shape


(312, 23)

In [25]:
table.head()

,EXPOSED,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,ANTENNA_ID_3,COUNT_3,ANTENNA_ID_4,...,COUNT_5,ANTENNA_ID_6,COUNT_6,ANTENNA_ID_7,COUNT_7,ANTENNA_ID_8,COUNT_8,ANTENNA_ID_9,COUNT_9,EPIDEMIC
USER,,,,,,,,,,,,,,,,,,,,,
FFFF78CB080636632B2D1DE7A7BFAB03,False,3415,21,2969,5,1320,3,2471,3,1115,...,65535,-1,65535,-1,65535,-1,65535,-1,65535,False
FFFF51FDDD55A2E24D074DE30C4798E9,True,1207,15,1171,14,1820,9,583,8,2026,...,4,1831,4,1625,2,401,1,825,1,False
FFFE1EDDA25369CBC467536FA6A787FD,True,1847,22,1753,14,1422,11,1028,10,765,...,9,862,8,2463,7,1416,6,871,5,True
FFFD0A912A959CDFCFFAB2F93BFBA435,True,1407,18,1702,8,1613,6,1704,5,1394,...,3,818,2,958,2,726,1,848,1,False
FFFCB235D6D917DDF350F5B11CCB0EE8,True,17,74,174,33,1123,4,1178,4,173,...,2,503,1,1039,1,1228,1,1327,1,True


In [27]:
len(np.unique(table.index.values)), table.shape

(3667451, (3667451, 22))

In [264]:
#mob_test = mini_sample['mobility_diameter']
results = mini_sample[(mini_sample['MOBILITY_DIAMETER']<10**3) & (mini_sample['ANTENNA_ID_8']>-1)].head()
print(results.shape)
for user in results.index:
    row = table.loc[user]
    row = row[row>-1]
    row = row[[ind for ind in row.index if "ANTENNA" in ind]]
    print(row)

(5, 23)
ANTENNA_ID_0    1149
ANTENNA_ID_1    1573
ANTENNA_ID_2    2812
ANTENNA_ID_3    2948
ANTENNA_ID_4      82
ANTENNA_ID_5     189
ANTENNA_ID_6     233
ANTENNA_ID_7     713
ANTENNA_ID_8    1477
ANTENNA_ID_9    1493
Name: 5D51048D591291F351EF09EADBD12F53, dtype: object
ANTENNA_ID_0      29
ANTENNA_ID_1    1700
ANTENNA_ID_2      73
ANTENNA_ID_3    2347
ANTENNA_ID_4      72
ANTENNA_ID_5    1235
ANTENNA_ID_6    1450
ANTENNA_ID_7    1561
ANTENNA_ID_8     674
ANTENNA_ID_9    1290
Name: 7BC884A1A97BEA2573B0BE18F7DB085C, dtype: object
ANTENNA_ID_0    1637
ANTENNA_ID_1    1352
ANTENNA_ID_2    1176
ANTENNA_ID_3    2026
ANTENNA_ID_4     268
ANTENNA_ID_5     942
ANTENNA_ID_6    1167
ANTENNA_ID_7    1255
ANTENNA_ID_8    1324
ANTENNA_ID_9    2968
Name: DD1ED3EB5C1AD91A7D590C5E1841F684, dtype: object
ANTENNA_ID_0    2043
ANTENNA_ID_1    1080
ANTENNA_ID_2    2323
ANTENNA_ID_3    2561
ANTENNA_ID_4    2269
ANTENNA_ID_5    2268
ANTENNA_ID_6    1097
ANTENNA_ID_7    1979
ANTENNA_ID_8    3152
ANTENNA_ID_

In [265]:
#testeamos que todos los users con 1 antenna tengan result ==0 
mob_test = mini_sample['MOBILITY_DIAMETER']
results = mini_sample[(mini_sample['MOBILITY_DIAMETER']<10) & (mini_sample['ANTENNA_ID_1']==-1)]
#mob_test[(mob_test.values<10) & (mini_sample['ANTENNA_ID_1']==-1)]
(results['MOBILITY_DIAMETER'].values==0).all()


True

### End mobility diameter result tests, Save table

In [269]:
mini_sample[[col for col in mini_sample.columns if ("ANTENNA" in col or "MOBILITY" in col)]].sample(10)

,ANTENNA_ID_0,ANTENNA_ID_1,ANTENNA_ID_2,ANTENNA_ID_3,ANTENNA_ID_4,ANTENNA_ID_5,ANTENNA_ID_6,ANTENNA_ID_7,ANTENNA_ID_8,ANTENNA_ID_9,MOBILITY_DIAMETER
USER,,,,,,,,,,,
9AF7A02045CB182FE90A88C9D03130AB,2178,392,2010,709,1407,1412,2084,520,2041,1083,1368.713640
45AEB0A267D2BFFD3983EF2C95CA206B,2964,2943,2301,3673,1012,3790,634,3681,1463,2404,2325.646079
3243E73992D14766D85D1F282F4C7690,291,2596,561,655,887,3143,3577,-1,-1,-1,1594.044422
32F7FFA33F10A88CEDEFE8D9BD4060F0,20,22,31,-1,-1,-1,-1,-1,-1,-1,707.770266
2A50CA984894ADD3989C8920D4B479AE,400,161,2322,1216,1326,1292,1565,637,1088,1205,1716.188192
A81BC0F14993A91B60E780236F7EBB0E,2175,780,911,1820,1766,1991,3199,1043,1649,1938,1534.150943
B8C676F2420B18EB3794FD4E8CAFC2BA,1247,1244,564,1697,307,2788,197,502,609,1171,1549.084125
168F7B598F3AD64679C123DFAC59D9B2,2746,218,-1,-1,-1,-1,-1,-1,-1,-1,524.782360
96397BB4600DE9D8440E0F1A77E30740,1637,1758,1438,1990,1324,1385,1750,1429,1601,1621,1447.386187


In [318]:
table.to_csv(output_file, 
             index = True,
            header = True,
            compression='gzip')

In [19]:
!du -ha $output_file*

173M	/home/juan/mobility-study/output2.txt.gz


In [323]:
table.sample(20)

,EXPOSED,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,ANTENNA_ID_3,COUNT_3,ANTENNA_ID_4,...,ANTENNA_ID_6,COUNT_6,ANTENNA_ID_7,COUNT_7,ANTENNA_ID_8,COUNT_8,ANTENNA_ID_9,COUNT_9,EPIDEMIC,MOBILITY_DIAMETER
USER,,,,,,,,,,,,,,,,,,,,,
18113EA238478375460839F4F5F3B017,True,1083,10,798,5,1608,5,248,4,849,...,2679,3,369,2,1016,1,1245,1,True,1596.220319
4C21FDD80CD47E153298C70D3527F079,True,3183,37,2875,31,2298,26,1449,16,2299,...,2296,4,2745,4,1154,3,1258,2,False,2847.255740
5B69FC601B10AD78AF0E250899DA4428,True,989,8,1379,8,327,7,990,6,1431,...,1610,3,988,1,1074,1,1098,1,True,737.250275
3104F21B0C633119F9EB28433F5DF0AC,True,2277,13,1990,5,1758,2,-1,65535,-1,...,-1,65535,-1,65535,-1,65535,-1,65535,True,739.327422
794E3583897B762E37DB26CB29BF2C95,True,1461,27,3825,19,2895,18,1012,17,2584,...,2055,6,2275,6,3787,4,2043,3,True,2655.544887
D3B4C0718A0A4B8B3A0495B9256C1262,True,540,4,1315,4,2013,3,2014,3,763,...,2520,2,-1,65535,-1,65535,-1,65535,False,701.064183
D0085B84A7EB85956E996C0EF2201453,True,594,43,2276,27,515,20,1386,17,2316,...,2174,13,771,12,1319,10,1721,10,True,742.937402
B10EEF8023AA67CD510AF3F9BD0D1405,True,399,33,1508,20,1415,11,526,10,1483,...,873,6,1573,5,111,4,1356,4,False,559.340031
3ECB89A445B6C708219724E905EF17D6,True,197,9,2362,8,2084,4,2278,4,2197,...,391,1,484,1,572,1,709,1,False,1368.713640


##  In case of antenna epidemic tag change
En el case en que cambien la definicion de la zona epidemica, esto cambiaria la condicion de epidemicidad para c/ antenna

In [8]:
#seteamos el lugar de trabajo
rootdir="/home/juan/mobility-study/"
os.chdir(rootdir)
year = "2015"; 

input_file= rootdir +"output2.txt.gz"\
                .format(y=year,ms=month_start,me=month_end,
                       ds=day_start,de=day_end)
    
output_file = "/home/juan/mobility-study/output2"
if week_end == True:
    output_file = output_file + "_wkend"
if night_filter == True:
    output_file += "_ngtfilter"
if hash_map == True:
    output_file += "_user_hash_map"
output_file += ".txt.gz"

In [9]:
!zcat $input_file | head -n10

USER,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,ANTENNA_ID_3,COUNT_3,ANTENNA_ID_4,COUNT_4,ANTENNA_ID_5,COUNT_5,ANTENNA_ID_6,COUNT_6,ANTENNA_ID_7,COUNT_7,ANTENNA_ID_8,COUNT_8,ANTENNA_ID_9,COUNT_9,MOBILITY_DIAMETER,EXPOSED,EPIDEMIC
FFFF78CB080636632B2D1DE7A7BFAB03,3415,21,2969,5,1320,3,2471,3,1115,2,-1,65535,-1,65535,-1,65535,-1,65535,-1,65535,1063,0,0
FFFF51FDDD55A2E24D074DE30C4798E9,1207,15,1171,14,1820,9,583,8,2026,5,1066,4,1831,4,1625,2,401,1,825,1,520,1,0
FFFE1EDDA25369CBC467536FA6A787FD,1847,22,1753,14,1422,11,1028,10,765,9,3072,9,862,8,2463,7,1416,6,871,5,990,1,0
FFFD0A912A959CDFCFFAB2F93BFBA435,1407,18,1702,8,1613,6,1704,5,1394,3,1398,3,818,2,958,2,726,1,848,1,491,1,0
FFFCB235D6D917DDF350F5B11CCB0EE8,17,74,174,33,1123,4,1178,4,173,3,890,2,503,1,1039,1,1228,1,1327,1,1613,1,0
FFFC9EC4F662C44D3006BA69C9F0DE85,876,19,1737,6,1744,5,326,1,378,1,503,1,1118,1,2216,1,2477,1,2962,1,3004,1,1
FFFB2F18F4CCF253C06B581EA518A7A5,2543,13,704,7,2540,7,1453,6,1642,5,45,4,2393,4,

In [10]:
#primero leo el file de antennas
antennas= pd.read_csv('/home/juan/mobility-study/antennas_mexico.csv',sep = "|",header=0,index_col=0)
antennas.index.rename("ANTENNA_ID",inplace=True)

In [11]:
table = pd.read_csv(
                input_file,
                engine = 'c',
                sep = '|',
                header = 0,
                index_col=0,
                dtype = {'EXPOSED':np.bool,'EPIDEMIC':np.bool, 'MOBILITY_DIAMETER':np.float32,
                         'ANTENNA_ID_0':np.int16,'COUNT_0':np.uint16,'ANTENNA_ID_1':np.int16,'COUNT_1':np.uint16,
                         'ANTENNA_ID_2':np.int16,'COUNT_2':np.uint16,'ANTENNA_ID_3':np.int16,'COUNT_3':np.uint16,
                        'ANTENNA_ID_4':np.int16,'COUNT_4':np.uint16,'ANTENNA_ID_5':np.int16,'COUNT_5':np.uint16,
                        'ANTENNA_ID_6':np.int16,'COUNT_6':np.uint16,'ANTENNA_ID_7':np.int16,'COUNT_7':np.uint16,
                        'ANTENNA_ID_8':np.int16,'COUNT_8':np.uint16,'ANTENNA_ID_9':np.int16,'COUNT_9':np.uint16}
                )
table['MOBILITY_DIAMETER'] = table['MOBILITY_DIAMETER'].astype(int)
## ojo con setear los dtypes para las antennas en np.usigned int pues ahi me transforma todos los -1s en el max del dtype

In [ ]:
# elimino todas las columnas que quiero re-calcular
table.drop('EPIDEMIC',axis=1, inplace=True)
table.drop('EXPOSED',axis=1, inplace=True)

In [116]:
table.head()

,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,ANTENNA_ID_3,COUNT_3,ANTENNA_ID_4,COUNT_4,...,COUNT_6,ANTENNA_ID_7,COUNT_7,ANTENNA_ID_8,COUNT_8,ANTENNA_ID_9,COUNT_9,MOBILITY_DIAMETER,EPIDEMIC,EXPOSED
USER,,,,,,,,,,,,,,,,,,,,,
FFFF78CB080636632B2D1DE7A7BFAB03,3415,21,2969,5,1320,3,2471,3,1115,2,...,65535,-1,65535,-1,65535,-1,65535,1063,False,True
FFFF51FDDD55A2E24D074DE30C4798E9,1207,15,1171,14,1820,9,583,8,2026,5,...,4,1625,2,401,1,825,1,520,False,True
FFFE1EDDA25369CBC467536FA6A787FD,1847,22,1753,14,1422,11,1028,10,765,9,...,8,2463,7,1416,6,871,5,990,False,True
FFFD0A912A959CDFCFFAB2F93BFBA435,1407,18,1702,8,1613,6,1704,5,1394,3,...,2,958,2,726,1,848,1,491,True,False
FFFCB235D6D917DDF350F5B11CCB0EE8,17,74,174,33,1123,4,1178,4,173,3,...,1,1039,1,1228,1,1327,1,1613,False,True


In [118]:
antennas.head()

,LATITUDE,LONGITUDE,STATE,EPIDEMIC
ANTENNA_ID_9,,,,
2797,14.671,-92.372,Chiapas,0
3060,14.682,-92.155,Chiapas,0
3344,14.721,-92.424,Chiapas,0
3487,14.778,-92.179,Chiapas,0
1924,14.786,-92.366,Chiapas,0


In [119]:
antena_ids = np.unique(antennas.index.values)
user_antenna_ids = set(np.unique(table['ANTENNA_ID_0'].values)) | set(np.unique(table['ANTENNA_ID_1'].values)) |\
                    set(np.unique(table['ANTENNA_ID_2'].values)) | set(np.unique(table['ANTENNA_ID_3'].values)) |\
                set(np.unique(table['ANTENNA_ID_4'].values)) | set(np.unique(table['ANTENNA_ID_5'].values)) |\
        set(np.unique(table['ANTENNA_ID_6'].values)) | set(np.unique(table['ANTENNA_ID_7'].values))
if -1 in user_antenna_ids:
    user_antenna_ids.remove(-1)
user_antenna_ids  = np.asarray(list(user_antenna_ids))

print(np.in1d(user_antenna_ids,antena_ids).all(), np.in1d(antena_ids,user_antenna_ids).all() )

print(np.in1d(antena_ids,user_antenna_ids).sum(), len(antena_ids))

(True, False)
(4221, 4429)


In [120]:
antenna_cols = [col for col in table.columns if "ANTENNA_ID" in col]

for col in antenna_cols:
    antennas.index.name = col
    table = table.join(antennas['EPIDEMIC'].astype(int), on=col,rsuffix='_{0}'.format(col.split("_")[-1]))
#solo falta renombrar la columna que no tiene rsuffix
table = table.rename(columns={'EPIDEMIC':'EPIDEMIC_0'})

In [128]:
epi_cols = [col for col in table.columns if 'EPIDEMIC_' in col]
table['EXPOSED'] = (table[epi_cols].sum(axis=1)>=1).astype(int)
table['EPIDEMIC'] =  table['EPIDEMIC_0'].values
for col in epi_cols:
    table.drop(col,axis=1,inplace=True)

In [129]:
table.sample(20)

,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,ANTENNA_ID_3,COUNT_3,ANTENNA_ID_4,COUNT_4,...,COUNT_6,ANTENNA_ID_7,COUNT_7,ANTENNA_ID_8,COUNT_8,ANTENNA_ID_9,COUNT_9,MOBILITY_DIAMETER,EXPOSED,EPIDEMIC
USER,,,,,,,,,,,,,,,,,,,,,
ED96A156B5712B53C57CFF8E8ABC182D,2002,98,2204,40,1962,30,2353,23,1446,21,...,17,1443,15,1444,10,514,9,846,1,0
A53F04CCB9D4DE72D1D80A9695CA0F38,2040,57,2282,21,754,20,1279,14,1877,13,...,8,343,4,131,3,1328,3,1535,1,0
4921680006841C65C5960D6E9C31667F,2555,2,676,1,1002,1,3312,1,-1,65535,...,65535,-1,65535,-1,65535,-1,65535,2128,1,0
45690AA7F26E3FEF051AD1252E716CB8,125,13,1310,8,257,4,250,3,123,2,...,1,1018,1,1401,1,-1,65535,695,1,0
F1CBA36A18650AB67A10181F1232CB3D,2047,75,657,22,409,19,2359,19,256,18,...,16,2208,15,954,14,321,11,1007,1,0
FAF9615D48D09049AF12453F767E5B9E,1174,16,116,11,3053,7,2921,6,3054,5,...,3,2387,2,2716,2,3591,2,2850,1,1
A17D41FD01A77C21E2C73C85D3D6C865,2688,188,834,20,729,1,-1,65535,-1,65535,...,65535,-1,65535,-1,65535,-1,65535,912,0,0
84B9E30956F6A66675D17A15189F9CCB,2075,64,568,46,1821,36,1289,29,840,19,...,8,713,4,1960,4,2079,4,1056,1,0
6903CCA86378FDB0D1589C0D7FC3ACE3,566,5,276,4,945,4,1244,3,297,2,...,1,1125,1,1230,1,2082,1,959,1,0


In [130]:
#fast consistency check
(table['EXPOSED'] >= table['EPIDEMIC']).all()

True

In [151]:
test.drop_duplicates?

In [163]:
## individual samples check
n=1500
test = table[['ANTENNA_ID_0','EPIDEMIC']].sample(n).sort_values(by='ANTENNA_ID_0').drop_duplicates()
#el drop_duplicates es porque puede haber 2 usuarios con la misma antenna
test.head()


,ANTENNA_ID_0,EPIDEMIC
USER,,
69E679784251659B335C109AB9BDEE5C,1,0
1C74A9C234FAEC2953DD1E727DFBEB30,2,0
1689BA5694E0E9CA56325AE24721552A,6,0
08CC1463A441C353F8699BBDA7819BE9,8,1
5298821615BB5D5CF795BC6A5268924E,9,1


In [164]:
#comparo que la condicion epidemica sea igual para c/uno de los users una vez que sortie por las antenna_ids
#el sort_index es necesario pues son unicas las antennas
(antennas[np.in1d(antennas.index.values,test['ANTENNA_ID_0'].values)]['EPIDEMIC'].sort_index().values == 
 test.EPIDEMIC.values).all()

True

In [13]:
%%time
#save file back
table.to_csv(output_file,
             sep = "|",
             index = True,
            header = True,
            compression='gzip')

CPU times: user 1min 42s, sys: 352 ms, total: 1min 42s
Wall time: 1min 43s


In [172]:
#chequeo a ver como esta escrito
!zcat $output_file | head

USER,ANTENNA_ID_0,COUNT_0,ANTENNA_ID_1,COUNT_1,ANTENNA_ID_2,COUNT_2,ANTENNA_ID_3,COUNT_3,ANTENNA_ID_4,COUNT_4,ANTENNA_ID_5,COUNT_5,ANTENNA_ID_6,COUNT_6,ANTENNA_ID_7,COUNT_7,ANTENNA_ID_8,COUNT_8,ANTENNA_ID_9,COUNT_9,MOBILITY_DIAMETER,EXPOSED,EPIDEMIC
FFFF78CB080636632B2D1DE7A7BFAB03,3415,21,2969,5,1320,3,2471,3,1115,2,-1,65535,-1,65535,-1,65535,-1,65535,-1,65535,1063,0,0
FFFF51FDDD55A2E24D074DE30C4798E9,1207,15,1171,14,1820,9,583,8,2026,5,1066,4,1831,4,1625,2,401,1,825,1,520,1,0
FFFE1EDDA25369CBC467536FA6A787FD,1847,22,1753,14,1422,11,1028,10,765,9,3072,9,862,8,2463,7,1416,6,871,5,990,1,0
FFFD0A912A959CDFCFFAB2F93BFBA435,1407,18,1702,8,1613,6,1704,5,1394,3,1398,3,818,2,958,2,726,1,848,1,491,1,0
FFFCB235D6D917DDF350F5B11CCB0EE8,17,74,174,33,1123,4,1178,4,173,3,890,2,503,1,1039,1,1228,1,1327,1,1613,1,0
FFFC9EC4F662C44D3006BA69C9F0DE85,876,19,1737,6,1744,5,326,1,378,1,503,1,1118,1,2216,1,2477,1,2962,1,3004,1,1
FFFB2F18F4CCF253C06B581EA518A7A5,2543,13,704,7,2540,7,1453,6,1642,5,45,4,2393,4,